In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 12 15:52:20 2023

@author: mickymwiti
"""

import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import itertools
import warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import numpy as np


# Suppress warnings
warnings.filterwarnings("ignore")

def find_best_arima_params(train):
    # Define the p, d, and q parameters to take on any value between 0 and 2
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    best_aic = float('inf')
    best_param = None

    # Generate all different combinations of p, q and q triplets
    for param in pdq:
        try:
            model = ARIMA(train, order=param)
            model_fit = model.fit()
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_param = param
        except:
            continue
    return best_param, best_aic

# Load the dataset
df = pd.read_csv('Oil_Dataset.csv')
headers = df.iloc[1]
df = df[1:]
df.columns = headers
df['DATE'] = pd.to_datetime(df['DATE'], format='%d/%m/%Y')
df.sort_values(by='DATE', inplace=True)

# Set the date column as the index
df.set_index('DATE', inplace=True)

# split the dataframe into gasoline and diesel
df_gasoline = df[['DATE', 'GASOLINE']]
df_diesel = df[['DATE', 'DIESEL']]

# Time series decomposition for gasoline
decomposition = seasonal_decompose(df_gasoline['GASOLINE'], model='multiplicative')
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Time series decomposition for diesel
decomposition = seasonal_decompose(df_diesel['DIESEL'], model='multiplicative')
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Stationarity for gasoline
result = adfuller(df_gasoline['GASOLINE'])
print('Gasoline ADF Statistic:', result[0])
print('Gasoline p-value:', result[1])

# Stationarity for diesel
result = adfuller(df_diesel['DIESEL'])
print('Diesel ADF Statistic:', result[0])
print('Diesel p-value:', result[1])

# Identifying the order of differencing for gasoline
df_gasoline['gasoline_price_diff'] = df_gasoline['GASOLINE'] - df_gasoline['GASOLINE'].shift()
plt.plot(df_gasoline['gasoline_price_diff'])
plt.show()                     
                     
# Identifying the order of differencing for diesel
df_diesel['diesel_price_diff'] = df_diesel['DIESEL'] - df_diesel['DIESEL'].shift()
plt.plot(df_diesel['diesel_price_diff'])
plt.show()

# Identifying the order of the ARIMA model for gasoline
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(df_gasoline['gasoline_price_diff'].dropna())
plt.show()
plot_pacf(df_gasoline['gasoline_price_diff'].dropna())
plt.show()

# Identifying the order of the ARIMA model for diesel
plot_acf(df_diesel['diesel_price_diff'].dropna())
plt.show()
plot_pacf(df_diesel['diesel_price_diff'].dropna())
plt.show()

# Identifying the seasonal component for gasoline
plt.plot(df_gasoline.groupby(df_gasoline.index.month)['GASOLINE'].mean())
plt.show()       

# Identifying the seasonal component for diesel
plt.plot(df_diesel.groupby(df_diesel.index.month)['DIESEL'].mean())
plt.show()

# Split the data into train and test sets for gasoline
train_gasoline = df_gasoline[:int(0.8*(len(df_gasoline)))]
test_gasoline = df_gasoline[int(0.8*(len(df_gasoline))):]

# Split the data into train and test sets for diesel
train_diesel = df_diesel[:int(0.8*(len(df_diesel)))]
test_diesel = df_diesel[int(0.8*(len(df_diesel))):]

# Find the best ARIMA parameters for gasoline
best_param_gasoline, best_aic_gasoline = find_best_arima_params(train_gasoline['GASOLINE'])
print(f'Best Parameters for Gasoline: {best_param_gasoline}')
print(f'Best AIC for Gasoline: {best_aic_gasoline}')

# Find the best ARIMA parameters for diesel
best_param_diesel, best_aic_diesel = find_best_arima_params(train_diesel['DIESEL'])
print(f'Best Parameters for Diesel: {best_param_diesel}')
print(f'Best AIC for Diesel: {best_aic_diesel}')

# Fit the ARIMA model to the gasoline price
model_gasoline = ARIMA(train_gasoline['GASOLINE'], order=best_param_gasoline)
model_fit_gasoline = model_gasoline.fit(disp=0)

# Fit the ARIMA model to the diesel price
model_diesel = ARIMA(train_diesel['DIESEL'], order=best_param_diesel)
model_fit_diesel = model_diesel.fit(disp=0)

# Forecast the gasoline prices for the next 10 years
forecast_gasoline, stderr, conf_int = model_fit_gasoline.forecast(steps=120)

# Forecast the diesel prices for the next 10 years
forecast_diesel, stderr, conf_int = model_fit_diesel.forecast(steps=120)

# Print the forecasted prices for gasoline
print(forecast_gasoline)

# Print the forecasted prices for diesel
print(forecast_diesel)

# Print the mean squared error for gasoline
predictions_gasoline = model_fit_gasoline.predict(start=len(train_gasoline), end=len(train_gasoline)+len(test_gasoline)-1, typ='levels')
mse_gasoline = mean_squared_error(test_gasoline['GASOLINE'], predictions_gasoline)
mae_gasoline = mean_absolute_error(test_gasoline['GASOLINE'], predictions_gasoline)
r2_gasoline = r2_score(test_gasoline['GASOLINE'], predictions_gasoline)
print(f'Test MSE for Gasoline: {mse_gasoline}')
print(f'Test MAE for Gasoline: {mae_gasoline}')
print(f'R-Squared for Gasoline: {r2_gasoline}')

# Print the mean squared error for diesel
predictions_diesel = model_fit_diesel.predict(start=len(train_diesel), end=len(train_diesel)+len(test_diesel)-1, typ='levels')
mse_diesel = mean_squared_error(test_diesel['DIESEL'], predictions_diesel)
mae_diesel = mean_absolute_error(test_diesel['DIESEL'], predictions_diesel)
r2_diesel = r2_score(test_diesel['DIESEL'], predictions_diesel)
print(f'Test MSE for Diesel: {mse_diesel}')
print(f'Test MAE for Diesel: {mae_diesel}')
print(f'R-Squared for Diesel: {r2_diesel}')

# Create a date range for the forecasted prices for gasoline
one = 1
start_date = train_gasoline.index[-1] + timedelta(days=one)
end_date = start_date + timedelta(days=120)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a date range for the forecasted prices for diesel
one = 1
start_date = train_diesel.index[-1] + timedelta(days=one)
end_date = start_date + timedelta(days=120)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a dataframe with the forecasted prices for gasoline
forecast_df_gasoline = pd.DataFrame(date_range, columns=['Date'])
forecast_df_gasoline['Forecast'] = forecast_gasoline

# Create a dataframe with the forecasted prices for diesel
forecast_df_diesel = pd.DataFrame(date_range, columns=['Date'])
forecast_df_diesel['Forecast'] = forecast_diesel

# Plot the forecasted prices for gasoline
plt.plot(forecast_df_gasoline['Forecast'])
plt.show()

# Plot the forecasted prices for diesel
plt.plot(forecast_df_diesel['Forecast'])
plt.show()

# Save the evaluation metrics to a csv file for gasoline
eval_metrics_gasoline = {'MSE': mse_gasoline, 'MAE': mae_gasoline, 'R-Squared': r2_gasoline}
eval_df_gasoline = pd.DataFrame.from_dict(eval_metrics_gasoline, orient='index', columns=['Value'])
eval_df_gasoline.to_csv('evaluation_metrics_gasoline.csv')

# Save the evaluation metrics to a csv file for diesel
eval_metrics_diesel = {'MSE': mse_diesel, 'MAE': mae_diesel, 'R-Squared': r2_diesel}
eval_df_diesel = pd.DataFrame.from_dict(eval_metrics_diesel, orient='index', columns=['Value'])
eval_df_diesel.to_csv('evaluation_metrics_diesel.csv')

# save the forecasted prices for gasoline and diesel to csv file
forecast_df_gasoline.to_csv('forecasted_prices_gasoline.csv',index=False)
forecast_df_diesel.to_csv('forecasted_prices_diesel.csv',index=False)          
                     
                     
                     
                     
                     
                     
                     
                     
                     
                    

KeyError: 'DATE'

In [12]:
df

,,,,,"(01/01/2016, 0.96, 0.95, 18,671,861, 3,949,015, 16,439,292)","1,304,885"
01/01/2016,0.96,0.95,"18,671,861","3,949,015","16,439,292","1,304,885"
01/02/2016,0.9,0.89,"17,350,331","3,157,953","14,740,817","1,218,465"
01/03/2016,0.97,0.93,"18,216,851","4,383,555","15,547,041","1,300,061"
01/04/2016,1.02,0.91,"15,965,471","4,218,647","14,069,660","1,269,557"
01/05/2016,1.07,0.96,"13,479,770","4,491,754","13,998,905","1,288,193"
...,...,...,...,...,...,...
01/06/2022,2.1,2.25,"20,244,676","3,853,814","17,995,938","2,146,999"
01/07/2022,1.89,2.01,"21,903,054","3,978,149","19,212,915","2,385,755"
01/08/2022,1.72,1.88,"22,076,930","3,937,020","18,898,396","2,345,473"
01/09/2022,1.59,1.9,"21,401,003","3,162,681","18,874,833","2,239,735"
